<a href="https://colab.research.google.com/github/AryanKohli88/Atrial-Fibrillation-predicting-model/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd
import numpy as np
from google.colab import drive
# drive.mount('/content/gdrive')
df = pd.read_csv('physionet_A_N.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18370 entries, 0 to 18369
Columns: 271 entries, TS_0 to CLASS
dtypes: float64(270), object(1)
memory usage: 38.0+ MB


In [67]:
df.isnull().sum() ## So we see there are no null values

TS_0      0
TS_1      0
TS_2      0
TS_3      0
TS_4      0
         ..
TS_266    0
TS_267    0
TS_268    0
TS_269    0
CLASS     0
Length: 271, dtype: int64

In [68]:
df.describe()

,TS_0,TS_1,TS_2,TS_3,TS_4,TS_5,TS_6,TS_7,TS_8,TS_9,...,TS_260,TS_261,TS_262,TS_263,TS_264,TS_265,TS_266,TS_267,TS_268,TS_269
count,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,...,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000,18370.000000
mean,6.460425,5.024823,4.541698,6.391889,6.158356,5.750136,4.165106,3.910996,3.444529,3.132281,...,7.016222,8.752041,8.110343,10.027055,7.581383,7.465705,9.962929,9.961132,8.139575,8.737180
std,272.334830,420.495452,402.284830,386.746853,378.025853,364.795591,347.348664,343.027604,337.468859,324.780727,...,198.537713,192.926446,197.262111,199.259975,196.717561,199.346913,199.376930,197.770150,197.604523,201.765898
min,-3614.000000,-6466.000000,-5019.000000,-4764.000000,-5952.000000,-6422.000000,-5255.000000,-6984.000000,-4687.000000,-5096.000000,...,-3479.000000,-2398.000000,-2790.000000,-3484.000000,-3402.000000,-3597.000000,-3756.000000,-3384.000000,-3154.000000,-4153.000000
25%,-71.000000,-79.000000,-79.000000,-79.000000,-79.000000,-77.000000,-77.000000,-75.000000,-75.000000,-75.000000,...,-60.000000,-59.000000,-59.000000,-59.000000,-60.000000,-60.000000,-59.000000,-59.000000,-59.000000,-58.000000
50%,-8.000000,-9.000000,-9.000000,-9.000000,-11.000000,-10.000000,-10.000000,-10.000000,-9.000000,-8.000000,...,-9.000000,-10.000000,-10.000000,-9.000000,-10.000000,-10.000000,-10.000000,-10.000000,-9.000000,-10.000000
75%,72.000000,82.000000,82.000000,80.000000,80.000000,79.000000,77.000000,73.000000,73.000000,70.750000,...,53.000000,52.000000,52.000000,53.000000,53.000000,52.000000,55.000000,55.000000,53.000000,53.000000
max,3344.000000,5514.000000,6134.000000,6305.000000,6479.000000,5658.000000,5728.000000,5847.000000,6070.000000,5308.000000,...,3109.000000,4482.000000,5790.000000,5405.000000,4082.000000,4027.000000,4170.000000,3503.000000,3908.000000,3190.000000


From the Above Description we conclude that the features are Catagorical as the final "CLASS" will either be "A" or "N".


In [69]:
dupcols = set()
for x in range(df.shape[1]):
  base_col = df.iloc[:,x]
  for y in range(x+1, df.shape[1]):
    comp_col = df.iloc[:,y]

    if base_col.equals(comp_col):
      dupcols.add(df.columns.values[y])

print(dupcols)

## hence we conclude there are no duplicate features in the given data

set()


# Analysing Features

In [70]:
df['Sum'] = df.sum(axis=1)
df['CLASS'].replace({'N':0 , "A":1}, inplace = True)
corr_matrix = df.corr()
(np.sqrt((corr_matrix['CLASS'])*(corr_matrix['CLASS']))).sort_values() 

<ipython-input-70-cf4cd9bff158>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df['Sum'] = df.sum(axis=1)


TS_162    0.000024
TS_237    0.000038
TS_57     0.000101
TS_101    0.000178
TS_35     0.000214
            ...   
TS_85     0.017930
TS_182    0.018926
TS_128    0.019335
TS_92     0.021560
CLASS     1.000000
Name: CLASS, Length: 272, dtype: float64

This Shows that a lot of features are not effecting the output a lot. We will drop features for whome magnitude of correlation outpur is less than 0.005

In [71]:
corr_matrix = corr_matrix.unstack()
filtered_matrix = corr_matrix[abs(corr_matrix)>0.005]
print(filtered_matrix['CLASS'])

TS_0      0.013327
TS_1      0.006802
TS_4     -0.007100
TS_6     -0.005801
TS_7      0.009615
            ...   
TS_260    0.005419
TS_263   -0.008422
TS_267   -0.012598
TS_269    0.005345
CLASS     1.000000
Length: 122, dtype: float64


In [72]:
X = df.drop(columns='CLASS', axis =1)
Y = df['CLASS']

In [73]:
from sklearn.model_selection import train_test_split
# del X_train
# del X_test
# del Y_train
# del Y_test
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state =42)

In [74]:
print(X.shape, X_train.shape, X_test.shape)

(18370, 271) (14696, 271) (3674, 271)


# Model Training

In [75]:
# # using logical regression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [76]:
# # using descision tree model
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import cross_val_score
# model = DecisionTreeRegressor()

In [77]:
model.fit(X_train, Y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

Hence, LogisticRegression model gives better results


# Model evaluation - Accuracy score

Accuracy on training set

In [78]:
from sklearn.metrics import accuracy_score
X_train_predict = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predict, Y_train)
print(training_data_accuracy)

0.8302259118127382


Accuracy on test set

In [79]:
from sklearn.metrics import accuracy_score
X_test_predict = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_predict, Y_test)
print(test_data_accuracy)

0.8206314643440392
